In [10]:
import requests
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import pymssql   

import pymssql
datacompany = (
                {'host':'192.168.250.40' , 'dbname':'IGU_NEW','user':'sa','password':'B1admin'} ,
                )
                
datalist=[]
listcom = []
for company in datacompany:
    
    msg_sql=  """
                  select          cardname,
                                year(balance ) iyear  ,
                                month(balance) imonth,
                                right(convert(varchar,balance,112),2) as idate
                        from 
                        (
                        select   
                                cardname ,
                        min( balance )balance
                        from (
                        SELECT year(a.docduedate) iyear,
                                month(a.docduedate) imonth,
                                right(convert(varchar,a.docduedate,112),2) iday,
                                b.cardname ,
                        min( a.docduedate ) balance
                        FROM OINV A
                        INNER JOIN OCRD B ON A.CARDCODE = B.CARDCODE 
                        INNER JOIN OCRG C ON B.GroupCode = C.GroupCode
                        where c.groupcode ='117'
                        group by year(a.docduedate) ,
                                month(a.docduedate) ,
                                right(convert(varchar,a.docduedate,112),2) ,
                                b.cardname 
                        having sum( a.DocTotal - a.PaidSys) <>0

                        union all
                        SELECT year(a.docduedate) iyear,
                                month(a.docduedate) imonth,
                                right(convert(varchar,a.docduedate,112),2) iday,
                                b.cardname ,
                        min( a.docduedate ) balance
                        FROM orin A
                        INNER JOIN OCRD B ON A.CARDCODE = B.CARDCODE 
                        INNER JOIN OCRG C ON B.GroupCode = C.GroupCode
                        where c.groupcode ='117'
                        group by year(a.docduedate) ,
                                month(a.docduedate) ,
                                right(convert(varchar,a.docduedate,112),2) ,
                                b.cardname 
                        having sum( a.DocTotal - a.PaidSys) <>0
                        )as a
                        group by  
                                cardname
) a                                                                                                                                                                                                                                                   
                """
   
    listcom.append(company["dbname"])
    conn = pymssql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)   
 
 
 
df.pivot_table(index=['cardname'],columns=["iyear","imonth"], values=["idate"],fill_value="-").to_excel("/data/cabang4.xlsx")
 

 

/tmp/ipykernel_35812/4255302575.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pandas.io.sql.read_sql(msg_sql,conn)
